In [1]:
import requests
from bs4 import BeautifulSoup as bs
from splinter import Browser
from pprint import pprint
import pandas as pd

In [9]:
##alt html TO DO using date concat
#http://www.espn.com/mens-college-basketball/scoreboard/_/group/100/date/20190324

html = 'http://www.espn.com/mens-college-basketball/scoreboard/_/group/100/date/20190324'

executable_path = {'executable_path': '/Users/APG/Desktop/messaround/resources/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [10]:
browser.visit(html)

In [11]:
soup = bs(browser.html, 'html.parser')

In [12]:
gs = soup.find_all('div', 'scoreboard-top no-tabs')

box_scores = []

game_id = 0

for g in gs:
    
#     find out if game is final  ----
    
    get_status = g.find('th','date-time')
    
#     print(get_status.text)
    if get_status.text == 'Final':
        
#    if game is final do  work ------
        
        get_box = g.find('table')
        
        print(get_box)

        team_names = get_box.find_all('span','sb-team-short')
        scores = get_box.find_all('td','total')
        
#         LENGTH CHECK
#         print('*',len(team_names))
#         print('*',len(scores))

#         print('******************')
#         for team_name in team_names:
#             print(team_name.text)
#         for score in scores:
#             print(score.text)
            
#         print('******************')
        
        for (team_name, score) in zip(team_names,scores):
            
            total = score.text
            box_scores.append(dict(school = team_name.text,
                                   total = total,
                                   last_number = total[-1],
                                   game_id = game_id                                 
                ))
            
                
#             print(school)
#             print(total)
#             print(game_id)
            
#         game_identifier  ---- outside inner loop
            
        game_id += 1
        

        

<table>
<thead>
<tr class="sb-linescore">
<th class="date-time">Final</th>
<th class="score ">1</th> <th class="score ">2</th>
<th class="total">T</th>
</tr>
</thead>
<tbody id="teams">
<tr class="away">
<td class="away">
<div class="img-container">
<a href="/mens-college-basketball/team/_/id/2116/ucf-knights" name="&amp;lpos=ncb:scoreboard:team">

</a>
</div>
<div class="sb-meta">
<h2 class="">
<span class="rank">9</span>
<a href="/mens-college-basketball/team/_/id/2116/ucf-knights" name="&amp;lpos=ncb:scoreboard:team">
<span class="sb-team-short">UCF</span>
<span class="sb-team-abbrev">UCF</span>
</a>
</h2>
<div class="record-container">
<p class="record">
		(24-9)
	</p>
<p class="record overall">24-9</p>
</div>
</div>
</td>
<td class="score">36</td>
<td class="score">40</td>
<td class="total"><span>76</span></td>
</tr>
<tr class="home">
<td class="home">
<div class="img-container">
<a href="/mens-college-basketball/team/_/id/150/duke-blue-devils" name="&amp;lpos=ncb:scoreboard:team"

In [13]:
#put into a dataframe
scoresheet = pd.DataFrame(box_scores)
print(scoresheet.head())

   game_id last_number      school total
0        0           6         UCF    76
1        0           7        Duke    77
2        1           1    Oklahoma    51
3        1           3    Virginia    63
4        2           9  Washington    59


In [14]:
#get game_ids into unique list
gamelist = scoresheet['game_id'].unique().tolist()
gamelist 

[0, 1, 2, 3, 4, 5, 6]

In [17]:
   
winning_combos = []

for g_id in gamelist:

    game_specific_df = scoresheet[scoresheet['game_id']==g_id]
   
    #final score = gwt # last digit of final = winkey #win_school = winning school name
    gamewinner = game_specific_df[game_specific_df['total']==game_specific_df['total'].max()]
    gwt = gamewinner['total'].tolist()[0]
    print(gwt)
  
    winkey = gamewinner['last_number'].tolist()[0]
    win_school = gamewinner['school'].tolist()[0]
    
    gameloser = game_specific_df[game_specific_df['total']==game_specific_df['total'].min()]
    glt = gameloser['total'].tolist()[0]
    print(glt)
    print('/\ score \/ details')
    
    loserkey = gameloser['last_number'].tolist()[0]
    lose_school = gameloser['school'].tolist()[0]
        
    print(gamewinner)
    print('****', winkey)
    
    print(gameloser)
    print('****', loserkey)
    
    winning_combos.append(dict(winner = winkey,
                               winning_total = gwt,
                               winning_school = win_school,
                               loser = loserkey,
                               losing_total = glt,
                               losing_school = lose_school))
    
        

77
76
/\ score \/ details
   game_id last_number school total
1        0           7   Duke    77
**** 7
   game_id last_number school total
0        0           6    UCF    76
**** 6
63
51
/\ score \/ details
   game_id last_number    school total
3        1           3  Virginia    63
**** 3
   game_id last_number    school total
2        1           1  Oklahoma    51
**** 1
81
59
/\ score \/ details
   game_id last_number          school total
5        2           1  North Carolina    81
**** 1
   game_id last_number      school total
4        2           9  Washington    59
**** 9
78
58
/\ score \/ details
   game_id last_number      school total
7        3           8  Texas Tech    78
**** 8
   game_id last_number   school total
6        3           8  Buffalo    58
**** 8
74
59
/\ score \/ details
   game_id last_number   school total
9        4           4  Houston    74
**** 4
   game_id last_number      school total
8        4           9  Ohio State    59
**** 9
67
58
/\ sco

In [18]:
pprint(winning_combos)

[{'loser': '6',
  'losing_school': 'UCF',
  'losing_total': '76',
  'winner': '7',
  'winning_school': 'Duke',
  'winning_total': '77'},
 {'loser': '1',
  'losing_school': 'Oklahoma',
  'losing_total': '51',
  'winner': '3',
  'winning_school': 'Virginia',
  'winning_total': '63'},
 {'loser': '9',
  'losing_school': 'Washington',
  'losing_total': '59',
  'winner': '1',
  'winning_school': 'North Carolina',
  'winning_total': '81'},
 {'loser': '8',
  'losing_school': 'Buffalo',
  'losing_total': '58',
  'winner': '8',
  'winning_school': 'Texas Tech',
  'winning_total': '78'},
 {'loser': '9',
  'losing_school': 'Ohio State',
  'losing_total': '59',
  'winner': '4',
  'winning_school': 'Houston',
  'winning_total': '74'},
 {'loser': '8',
  'losing_school': 'Liberty',
  'losing_total': '58',
  'winner': '7',
  'winning_school': 'Virginia Tech',
  'winning_total': '67'},
 {'loser': '4',
  'losing_school': 'UC Irvine',
  'losing_total': '54',
  'winner': '3',
  'winning_school': 'Oregon',


In [19]:
mmad = pd.read_excel('./resources/marchmad.xlsx')
mmad

,8,2,3,1,9,0,7,5,4,6
4,Wayne,Justin Grohnke,Big Sausage,Sweat Melons,Big Cheese,Big Cheese,Joe Eagles,DeCarlo Family,Sal & Lizzy,42
2,Seibert #11,GRAT,AZ Boyz,K. Becker,42,Jim O'C,Pag Boyz,Mic M,Al Jones,Scott Boyens
7,SMO,Baby Boyz,42,Mike & Mike,Mike O,Joe S,GRAT,Mom 7,Jeff M,Marissa's Dad
3,E.S.,Fire Fly,Browny,FURM 2,Peter M. Garetto,Buck 1,Z,Dave and TT,Juan Alcala,Rodney L. (Al Jones)
9,FURM 1,Pat M,SMO #2,Ochoa,Brad & Priscilla,BI Erin,AL Jones,Julie Smevoll,Amanda C,Gonzo
8,Sharon OF,EJVB,Joe S,Allan (Grohnke),AZ Boyz,Peter Blue,Anthony G,Saint,E.S.,Carlos Gutierrez
0,EL Chapo,Z,Jim O'C,Kim and John,GRAT,Eric C,42,FURM 3,Pat M,DeCarlo Family
1,Scott Boyens,Gonzo,Heslop,Ronnie G,Gina Beggars,PE3,AL Jones,GRAT,Pag Boyz,Orozco
6,BJ 1,Peter Blue,Jeff M,Al Jones,BI Erin,Heslop,Jeff M,Angelo G,AZ Boyz,Buck 2
5,Tom Tucker,RJ (Grohnke),Victor C,Jonathan,Ray & Di,BJ 2,Angelo G,Cantelo,Labo,Whitey & Mary


In [20]:
#change fundraiser amount @TO DO make this key up with round totals, need to get from organizer
out = '$25'

for i in winning_combos:
    win = int(i['winner'])
    lose = int(i['loser'])
    cashprize = mmad.at[lose,win]
    i['square'] = cashprize
    i['payout'] = out
    print("*******")
    print(f"{i['winning_school']} beat {i['losing_school']}")

    print(f"{cashprize} won the square")
    print("*******")

*******
Duke beat UCF
Jeff M won the square
*******
*******
Virginia beat Oklahoma
Heslop won the square
*******
*******
North Carolina beat Washington
Ochoa won the square
*******
*******
Texas Tech beat Buffalo
Sharon OF won the square
*******
*******
Houston beat Ohio State
Amanda C won the square
*******
*******
Virginia Tech beat Liberty
Anthony G won the square
*******
*******
Oregon beat UC Irvine
Big Sausage won the square
*******


In [21]:
winning_combos

[{'winner': '7',
  'winning_total': '77',
  'winning_school': 'Duke',
  'loser': '6',
  'losing_total': '76',
  'losing_school': 'UCF',
  'square': 'Jeff M',
  'payout': '$25'},
 {'winner': '3',
  'winning_total': '63',
  'winning_school': 'Virginia',
  'loser': '1',
  'losing_total': '51',
  'losing_school': 'Oklahoma',
  'square': 'Heslop',
  'payout': '$25'},
 {'winner': '1',
  'winning_total': '81',
  'winning_school': 'North Carolina',
  'loser': '9',
  'losing_total': '59',
  'losing_school': 'Washington',
  'square': 'Ochoa',
  'payout': '$25'},
 {'winner': '8',
  'winning_total': '78',
  'winning_school': 'Texas Tech',
  'loser': '8',
  'losing_total': '58',
  'losing_school': 'Buffalo',
  'square': 'Sharon OF',
  'payout': '$25'},
 {'winner': '4',
  'winning_total': '74',
  'winning_school': 'Houston',
  'loser': '9',
  'losing_total': '59',
  'losing_school': 'Ohio State',
  'square': 'Amanda C',
  'payout': '$25'},
 {'winner': '7',
  'winning_total': '67',
  'winning_school'

In [22]:
output_df = pd.DataFrame(winning_combos)

output = output_df[['losing_school', 'losing_total', 'winning_school', 'winning_total', 'loser', 'winner', 'square', 'payout']]

output

,losing_school,losing_total,winning_school,winning_total,loser,winner,square,payout
0,UCF,76,Duke,77,6,7,Jeff M,$25
1,Oklahoma,51,Virginia,63,1,3,Heslop,$25
2,Washington,59,North Carolina,81,9,1,Ochoa,$25
3,Buffalo,58,Texas Tech,78,8,8,Sharon OF,$25
4,Ohio State,59,Houston,74,9,4,Amanda C,$25
5,Liberty,58,Virginia Tech,67,8,7,Anthony G,$25
6,UC Irvine,54,Oregon,73,4,3,Big Sausage,$25


In [ ]:
output.to_excel('results.xlsx')